# Fine tuning with LoRA

### This code is underdeveloped as the idea of PEFT has been dropped in this research due to the complexity of the customized training process, leading to no improvements in both memory and time complexity. This code runs, but has not been maintained and might contain minor bugs.

In [ ]:
from transformers import pipeline
import torch
import numpy as np

device = (
    torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
)
# if it is not MPS, try CUDA
print(torch.__version__)
print(torch.cuda.is_available())
device = torch.device("cuda") if torch.cuda.is_available() else device
print(f"Using device: {device}")


In [3]:
def set_seed(seed):
    torch.manual_seed(seed)
    # random.seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using GPUs

set_seed(4242)

### Task 1: Sentiment Analysis
Sentiment Analysis is the task of classifying a given text into positive, negative, or neutral sentiments.

In this example, we will use a pre-trained model for sentiment analysis. The `pipeline` will automatically download and load a model that has been pre-trained on a large dataset to perform this task.

In [ ]:
#model_name = "ncbi/MedCPT-Article-Encoder"
model_name = "bert-base-uncased"


classifier = pipeline("sentiment-analysis", model=model_name, device=device)
result = classifier(
    "The development of a recombinant polyclonal antibody therapy for COVID-19 by GigaGen represents an early-stage positive news in response to a global health crisis. However, such initiatives often come with high risk and uncertainty given the complexity and time required for clinical trials and approval processes. Additionally, competition in the COVID-19 treatment space is intense, with many companies pursuing similar therapies. These factors make it essential to remain cautious, monitoring further developments and data closely."
)

print(f"Sentiment Analysis Result: {result}")

# 3. Fine-Tuning Pre-trained Models
While the pre-trained models provided by Hugging Face are powerful, you may want to fine-tune them for a specific task or dataset.

Fine-tuning involves taking a pre-trained model and training it further on your own data. This can improve the model’s performance for specific use cases.



### Step 1: Load Dataset

In [8]:
from datasets import load_dataset, Features, ClassLabel, Value
import os, sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
data_file_path = os.path.join(parentdir, "data/processed/finetuning_5_labels_topic_pruned.csv") # choose the nr of labels urself
if not os.path.exists(data_file_path):
    raise FileNotFoundError(f"The file {data_file_path} does not exist!")

In [6]:
# ALTERNATIVE FOR COLLAB
from datasets import load_dataset, Features, ClassLabel, Value
data_file_path = "finetuning_5_labels_topic_pruned.csv"

In [ ]:
# Define the features
features = Features(
    {
        "text": Value("string"),
        "labels": ClassLabel(names=["0.0", "1.0", "2.0", "3.0", "4.0"]),
    }
)

# Load the dataset with the specified features
dataset = load_dataset(
    "csv",
    data_files=data_file_path,
    features=features,
)

**Train Test Split**

In [10]:
#add a columns "id" with a unique row id
dataset["train"] = dataset["train"].add_column("id", range(len(dataset["train"])))

In [ ]:
print(dataset)
print(dataset["train"][1])

In [12]:
from sklearn.model_selection import train_test_split

labels = np.array(dataset["train"]["labels"])

# Generate indices for stratified split
indices = np.arange(len(labels))
train_indices, test_indices = train_test_split(
    indices,
    test_size=0.1,
    stratify=labels,
    random_state=42,  # Ensures reproducibility
)

# Create stratified train and test datasets, remember its a pandas dataframe
train_dataset = dataset["train"].select(train_indices)
test_dataset = dataset["train"].select(test_indices)

In [ ]:
test_dataset[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming labels is a numpy array of your dataset labels
labels = np.array(dataset["train"]["labels"])
label_counts = np.bincount(labels)
label_percentages = label_counts / label_counts.sum() * 100 
label_names = ["Strongly Negative","Negative", "Neutral", "Positive", "Strongly Positive"]
plt.bar(range(len(label_percentages)), label_percentages)
plt.xlabel("Label")
plt.ylabel("Percentage (%)") 
plt.title("Percentage Distribution of Labels")  
plt.xticks(range(len(label_percentages)), label_names)  
plt.show()


In [ ]:
#now plot the distribution of text lengths by words

text_lengths = [len(text.split()) for text in dataset["train"]["text"]]
plt.hist(text_lengths, bins=20)
plt.xlabel("Text Length (Words)")
plt.ylabel("Frequency")
plt.title("Distribution of Text Lengths")   

### Step 2: Tokenize the Dataset
The dataset needs to be tokenized before it can be fed into the model. Tokenization converts the text data into numerical format (tokens) that the model can process.

We'll use the `AutoTokenizer` class from HuggingFace to tokenize the data. The `AutoTokenizer` class automatically selects the appropriate tokenizer for the model based on the `model_name`.

Tokenization or transformation of the dataset can be done using the `map` method, which applies a function to all the elements of the dataset. This is easily done by defining a function that tokenizes the text data and then applying it to the dataset. When `batched=True`, the function will be applied to batches of data, which can improve performance by applying the function in parallel.

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer, padding="max_length", max_length=512
)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)


tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_test


### Step 3: Load a Pre-trained Model
Now that the data is tokenized, we'll load a pre-trained model that we'll fine-tune for sentiment classification.

We need to import `AutoModelForSequenceClassification` for that. The key feature of this class is that it adds a classification head on top of the pre-trained transformer model to allow it to classify sequences into one or more categories (e.g., positive/negative sentiment, spam/ham, etc.). The `from_pretrained` method loads the pre-trained model with the specified configuration. The `num_labels` parameter specifies the number of labels in the classification task (binary in this case).

In [ ]:
from transformers import AutoModelForSequenceClassification

lora_model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=5, # ignore_mismatched_sizes=True
).to(device)


### Step 4: Set Up the Trainer
Hugging Face provides the Trainer class to help with the training and fine-tuning of models. We need to set up the trainer by providing the model, training arguments, and the datasets.


In [19]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="weighted", labels=[0, 1, 2, 3, 4]
    )
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(dataset["train"]["labels"]),
    y=dataset["train"]["labels"],
)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print("Class Weights:", class_weights)

***Loss function***

In [21]:
import torch.nn as nn
import torch.nn.functional as F


# Define custom loss function
class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights):
        super(WeightedCrossEntropyLoss, self).__init__()
        self.class_weights = class_weights

    def forward(self, logits, labels):
        return F.cross_entropy(logits, labels, weight=self.class_weights)

**System metric tracking**

In [22]:
# Helper functions for tracking system metrics during training
import time
import psutil
import subprocess
import os

def get_gpu_memory_usage():
    # Run nvidia-smi to get both used and total memory
    result = subprocess.run(
        ["nvidia-smi", "--query-gpu=memory.used,memory.total", "--format=csv,nounits,noheader"],
        stdout=subprocess.PIPE
    )
    used_memory, total_memory = result.stdout.decode("utf-8").strip().split("\n")[0].split(", ")

    return int(used_memory), int(total_memory)

# Pass the model type as a string
def track_performance(model_type):
    process = psutil.Process(os.getpid())

    start_time = time.time()

    trainer.train()

    # Track memory and time
    memory_usage = process.memory_info().rss / (1024 * 1024)
    gpu_memory_used, gpu_memory_total = get_gpu_memory_usage()
    end_time = time.time()

    time_elapsed = end_time - start_time

    print(f"{model_type} - Time taken: {time_elapsed:.2f} seconds")
    print(f"{model_type} - CPU Memory usage: {memory_usage:.2f} MB")
    print(f"{model_type} - GPU Memory usage: {gpu_memory_used} MB / {gpu_memory_total} MB")

#### Defining the custom trainer

In [23]:
from transformers import Trainer, TrainingArguments
from torch.optim.lr_scheduler import OneCycleLR

# Define custom trainer
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights, optimizer=None, num_training_steps=None, total_epochs=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        self.optimizer = optimizer
        self.num_training_steps = num_training_steps
        self.total_epochs = total_epochs

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # Added **kwargs to handle required arguments unused by our loss
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = WeightedCrossEntropyLoss(self.class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

    def create_scheduler(self, num_training_steps=None, optimizer=None): # Slanted Triangular Learning Rate
        if self.lr_scheduler is None:
            num_training_steps = self.num_training_steps or (
                len(self.train_dataset) // self.args.per_device_train_batch_size * self.args.num_train_epochs
            )
            self.lr_scheduler = OneCycleLR(
                optimizer=optimizer,
                max_lr=5e-5,  # Peak learning rate
                total_steps=num_training_steps,
                pct_start=0.3,  # Fraction of steps to increase LR for
                anneal_strategy='cos',  # Linear decrease after peak
            )
        return self.lr_scheduler

***Defining the training callback***

In [24]:
from transformers import TrainerCallback

class TrainingMonitorCallback(TrainerCallback):
    """ Tracks the learning rate and gradual unfreezing"""
    def __init__(self, model, total_layers, plot_save_path=None):
        super().__init__()
        self.model = model
        self.total_layers = total_layers
        self.lrs = []
        self.plot_save_path = plot_save_path

    def on_step_end(self, args, state, control, optimizer, **kwargs):
        """Called at the end of each training step"""
        if optimizer is not None:  # Safety check
            lr = optimizer.param_groups[0]['lr']
            self.lrs.append(lr)
            
    def plot_learning_rate(self):
        """Plot the learning rate history"""
        if len(self.lrs) > 0:
            plt.figure(figsize=(10, 6))
            plt.plot(self.lrs)
            plt.title('Learning Rate over Training Steps')
            plt.xlabel('Training Step')
            plt.ylabel('Learning Rate')
            plt.grid(True)
            if self.plot_save_path:
                plt.savefig(self.plot_save_path)
            else:
                plt.show()

#### Trainer Definition

In [ ]:
from peft import LoraConfig, get_peft_model


lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="all",
    task_type="SEQ_CLS"  # sequence classification
)

# Wrap the model with LoRA configuration
lora_model = get_peft_model(lora_model, lora_config)
lora_model.print_trainable_parameters()

In [27]:
# Create the callback
tracker = TrainingMonitorCallback(lora_model, total_layers=12)

training_args = TrainingArguments(
    output_dir="./results",  # Output directory
    eval_strategy="epoch",  # Evaluate after each epoch
    save_strategy="epoch",
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,  # Batch size for evaluation
    gradient_accumulation_steps=2,  # Gradient accumulation steps FOR MEMORY EFFICIENCY
    num_train_epochs=10,  # Number of epochs
    load_best_model_at_end=True,  # Load the best model at the end
    metric_for_best_model="f1",  # Use F1 score to select the best model
    save_total_limit=1,  # Limit the total amount of checkpoints
    report_to="none",  # Disable all integrations, including wandb
)

num_training_steps = (
    (len(tokenized_train) // training_args.per_device_train_batch_size)
    // training_args.gradient_accumulation_steps
) * training_args.num_train_epochs


custom_optimizer = torch.optim.AdamW(lora_model.parameters(), lr=2e-5, weight_decay=0.001)

trainer = CustomTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_train, 
    eval_dataset=tokenized_test,  
    compute_metrics=compute_metrics,
    class_weights=class_weights,  
    data_collator=data_collator,
    optimizer=custom_optimizer,
    num_training_steps=num_training_steps,
    total_epochs=training_args.num_train_epochs,
)

trainer.optimizer = custom_optimizer
trainer.optimizer = custom_optimizer

### Step 5: Fine-tune the Model
Now that the trainer is set up, we can start the fine-tuning process.

Run the following cell to fine-tune the model.

In [ ]:
# Learn without gradual unfreezing
track_performance("LoRA")

### Step 6: Evaluate the Model
After training, we can evaluate the model’s performance on the test set.

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

In [ ]:
tracker.plot_learning_rate()

### Step 7: Try out model

In [ ]:
input_string = "BioVie Announces Alignment with FDA on Clinical Trial to Assess Bezisterim in Parkinson’s Disease,SUNRISE-PD to evaluate the effect of bezisterim (NE3107) on motor and non-motor symptoms in ~60 patients with Parkinson’s disease who are naïve to carbidopa/levodopa,SUNRISE-PD to evaluate the effect of bezisterim (NE3107) on motor and non-motor symptoms in ~60 patients with Parkinson’s disease who are naïve to carbidopa/levodopa"

# Tokenize the input string
inputs = tokenizer(input_string, return_tensors="pt").to(device)

# Get predictions (logits)
with torch.no_grad():  # Disable gradient computation since we're just doing inference
    outputs = lora_model(**inputs)
    logits = outputs.logits
print(logits)
predicted_label = torch.argmax(logits, dim=1).item()


print(f"Predicted label: {predicted_label}")

### Step 8. Saving the Fine-tuned Model
After training, it is often useful to save the fine-tuned model, so you can use it later without needing to re-train it.

In [ ]:
model.push_to_hub(
    f"akseljoonas/MedCPT-Article-Encoder-finetuned-{eval_results['f1']:.3f}"
)
tokenizer.push_to_hub(
    f"akseljoonas/MedCPT-Article-Encoder-finetuned-{eval_results['f1']:.3f}"
)